In [16]:
import logging
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from markdownify import markdownify as md
import time
from openai import OpenAI
import os
from dotenv import load_dotenv
from pydantic import BaseModel
from typing import List, Optional

load_dotenv()

def setup_selenium():
    """Sets up Selenium with headless Chrome."""
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def fetch_html(driver, url, timeout=30):
    """Fetches HTML content from URL using Selenium."""
    try:
        driver.get(url)
        WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((By.CLASS_NAME, "note"))
        )
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        return driver.page_source
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return None

def extract_reviews_to_markdown(url):
    """Extracts the paper abstract and all reviews from a given OpenReview URL and returns them as markdown text."""
    driver = setup_selenium()
    try:
        html = fetch_html(driver, url)
        if not html:
            return "Error: Could not fetch page content"
        
        soup = BeautifulSoup(html, "html.parser")
        
        # Get paper title
        title = soup.find("h2", class_="citation_title")
        markdown_text = f"# {title.text.strip() if title else 'Unknown Paper'}\n\n"
        
        # Extract paper abstract
        abstract = ""
        abstract_header = soup.find("h4", string=re.compile(r"Abstract", re.I))
        if abstract_header:
            abstract_para = abstract_header.find_next_sibling("p")
            if abstract_para:
                abstract = abstract_para.text.strip()
        if abstract:
            markdown_text += f"## Abstract\n\n{abstract}\n\n"
        
        # Find all review notes
        reviews = []
        for note in soup.find_all("div", class_="note"):
            invitation = note.find("span", class_="invitation")
            if not invitation:
                continue
                
            invitation_text = invitation.text.strip()
            if "Official Review" in invitation_text:
                # Get reviewer
                signature = note.find("span", class_="signatures")
                reviewer = signature.text.strip() if signature else "Anonymous"
                
                # Get review content
                content = note.find("div", class_="note-content")
                if content:
                    review_text = md(str(content))
                    reviews.append((reviewer, review_text))
        
        # Format reviews as markdown
        for i, (reviewer, review) in enumerate(reviews, 1):
            markdown_text += f"## Review {i}\n\n"
            markdown_text += f"**Reviewer:** {reviewer}\n\n"
            markdown_text += f"{review}\n\n"
            markdown_text += "---\n\n"
        
        return markdown_text
    
    finally:
        driver.quit()
        
class ReviewQualityAnalyzer(BaseModel):
    language_quality_and_professionalism_paragraph: str
    language_quality_and_professionalism_score: int
    internal_consistency_paragraph: str
    internal_consistency_score: int
    constructiveness_of_criticism_paragraph: str
    constructiveness_of_criticism_score: int
    
client = OpenAI(
    api_key=os.getenv("GEMINI_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

# Example usage
url = "https://openreview.net/forum?id=Iyve2ycvGZ"  # Replace with your URL
markdown_output = extract_reviews_to_markdown(url)
# print(markdown_output)

response = client.beta.chat.completions.parse(
    model="gemini-2.0-flash-exp",
    n=1,
    messages=[
        {
            "role": "system", 
            "content": (
                "First generate the paragraph than given the paragraph generate the score. "
                "You are an expert review analysis assistant. You will be provided with a paper review "
                "in Markdown format. Your task is to evaluate this review comprehensively under three criteria: "
                "1) Language quality and professionalism, 2) Internal consistency, and 3) Constructiveness of criticism. "
                "For each criterion, create a detailed paragraph justifying your evaluation and assign a score between "
                "1 (lowest) and 10 (highest). Ensure your response corresponds to the structure of the ReviewQualityAnalyzer model."
            )
        },
        {
            "role": "user", 
            "content": f"{markdown_output}"
        }
    ],
    response_format=ReviewQualityAnalyzer
)

a = response.choices[0].message.content

print(a)

{
  "internal_consistency_score": 7,
  "constructiveness_of_criticism_paragraph": "The reviews offer a range of constructive criticisms, highlighting areas for improvement while acknowledging the strengths of the proposed method. Reviewer TqqW identifies the need for broader dataset testing and clearer resource usage comparisons, which are practical suggestions for strengthening the paper's impact. Reviewer pSht points out the weakness of the baseline comparison and the practical limitations of the dynamic programming algorithm, suggesting more relevant benchmarks and further discussion of optimization time. Reviewer GU1Q provides the most detailed critique, pinpointing missing implementation details, insufficient related work discussions, and a lack of qualitative comparisons. All reviews offer specific, actionable feedback aimed at refining both the methodology and the presentation of the research. Overall the reviewers constructively guide the authors in enhancing the paper's rigor 